# Q3

## Relationship between ship deadweight and emissions intensity

In [86]:
import pandas as pd 

In [118]:
df_sample = pd.read_csv('./EU_MRV_all_data.csv', nrows=10000, index_col=0)

In [120]:
df_sample.columns

Index(['IMO Number', 'Name', 'Ship type', 'Reporting Period',
       'Technical efficiency', 'Port of Registry', 'Home Port', 'Ice Class',
       'DoC issue date', 'DoC expiry date', 'Verifier Number', 'Verifier Name',
       'Verifier NAB', 'Verifier Address', 'Verifier City',
       'Verifier Accreditation number', 'Verifier Country', 'A', 'B', 'C', 'D',
       'Total fuel consumption [m tonnes]',
       'Fuel consumptions assigned to On laden [m tonnes]',
       'Total CO₂ emissions [m tonnes]',
       'CO₂ emissions from all voyages between ports under a MS jurisdiction [m tonnes]',
       'CO₂ emissions from all voyages which departed from ports under a MS jurisdiction [m tonnes]',
       'CO₂ emissions from all voyages to ports under a MS jurisdiction [m tonnes]',
       'CO₂ emissions which occurred within ports under a MS jurisdiction at berth [m tonnes]',
       'CO₂ emissions assigned to Passenger transport [m tonnes]',
       'CO₂ emissions assigned to Freight transport [m t

In [122]:
# try to pick out columns of interest
relevant_columns = ['Average density of the cargo transported [m tonnes / m³]',

                    'Total fuel consumption [m tonnes]',
                    'Total CO₂ emissions [m tonnes]',

                    'Annual average Fuel consumption per distance [kg / n mile]',
                    'Annual average Fuel consumption per transport work (dwt) [g / dwt carried · n miles]',

                    'Annual average CO₂ emissions per distance [kg CO₂ / n mile]',
                    'Annual average CO₂ emissions per transport work (mass) [g CO₂ / m tonnes · n miles]',
                    'Annual average CO₂ emissions per transport work (dwt) [g CO₂ / dwt carried · n miles]',

                    'CO₂ emissions per distance on laden voyages [kg CO₂ / n mile]',
                    'CO₂ emissions per transport work (mass) on laden voyages [g CO₂ / m tonnes · n miles]',
                    'CO₂ emissions per transport work (dwt) on laden voyages [g CO₂ / dwt carried · n miles]',
]

What is emissions intencity?

https://en.wikipedia.org/wiki/Emission_intensity

https://www.dnv.com/maritime/insights/topics/CII-carbon-intensity-indicator/index.html


'Annual average CO₂ emissions per distance [kg CO₂ / n mile]'?

or

CO₂ emissions per transport work (mass) ?

Deadweight is the sum of the weights of cargo, fuel, fresh water, ballast water, provisions, passengers, and crew.

Useful column: 'CO₂ emissions per transport work (dwt) on laden voyages [g CO₂ / dwt carried · n miles]',

https://www.diva-portal.org/smash/get/diva2:1549614/FULLTEXT01.pdf 

Thoughts:

Can I calculate total mass

Can I get mass from average density and volume

Can I calculate deadweight from dwt column

deadweight = total emissions / (emissions intensity (deadweight) * distance)

## Get distance

In [126]:
# Get Distance (Copy from Q4)

# Convert strings to nans
average_fuel_consuption_kg = pd.to_numeric(df_sample['Annual average Fuel consumption per distance [kg / n mile]'], errors='coerce')


# Convert m tonnes to kg
total_fuel_consumption_kg = df_sample['Total fuel consumption [m tonnes]'] * 1000

distance = total_fuel_consumption_kg / average_fuel_consuption_kg

print(f"Median: {distance.median()}")
print(f"Mean: {distance.mean()}")

Median: 22348.58924117193
Mean: 28168.330364733854


## Calculate deadweight with distance

In [124]:
# Calculate deadweight with average CO2 per dwt per distance
total_CO2_emissions_grammes = df_sample['Total CO₂ emissions [m tonnes]'] * 1000000
average_CO2_emissions_grammes_per_dwt_distance = pd.to_numeric(df_sample['Annual average CO₂ emissions per transport work (dwt) [g CO₂ / dwt carried · n miles]'], errors='coerce')
deadweight = total_CO2_emissions_grammes / (average_CO2_emissions_grammes_per_dwt_distance * distance)

print("Median:", deadweight.median())
print(deadweight.isna().value_counts())
# or...

Median: 6560.907195403154
True     8957
False    1043
dtype: int64


In [127]:
# ...or
# Calculate deadweight with average Fuel consumption per dwt per distance
total_fuel_consumption_grammes = total_fuel_consumption_kg * 1000
average_fuel_consumption_per_dwt_per_distance = pd.to_numeric(df_sample['Annual average Fuel consumption per transport work (dwt) [g / dwt carried · n miles]'], errors='coerce')
print(average_fuel_consumption_per_dwt_per_distance.isna().mean())
deadweight = total_fuel_consumption_grammes / (average_fuel_consumption_per_dwt_per_distance * distance)

print("Median:", deadweight.median())
print(deadweight.isna().value_counts())

0.8957
Median: 6564.760793465577
True     8957
False    1043
dtype: int64


In [100]:
deadweight_inliers = deadweight[deadweight.between(deadweight.quantile(.01), deadweight.quantile(.99))] # without outliers
fig = px.box(x=deadweight_inliers, labels={"x":"Deadweight"})
fig.show()

## Plot relationship between deadwieght and CO2 per distance

In [117]:
import plotly.express as px
CO2_per_distance_m_tonnes = pd.to_numeric(df_sample['Annual average CO₂ emissions per distance [kg CO₂ / n mile]'], errors='coerce') / 1000

fig = px.scatter(x=deadweight, y=CO2_per_distance_m_tonnes, color=df_sample['Ship type'], range_x=[0, deadweight.quantile(0.9)], range_y=[0, CO2_per_distance_m_tonnes.quantile(0.9)], labels={"x":"Deadweight", "y":"CO₂ emissions per distance [m tonne CO₂ / n mile]"}, height=600, width=1000, title="Relatonship between Deadweight and Carbon Intensity", trendline="lowess", trendline_scope="overall", trendline_color_override="black")
fig.show()